# Test air_temperature forecast implementation


In [790]:
# install the modules on the OS
#!pip install influxdb

# import the modules
import pandas as pd
import datetime as dt
#from influxdb import DataFrameClient
import numpy as np
import influxdb
import os
import math
from decimal import *

In [791]:
# define the database connection string
DB_HOST = 'localhost' 
DB_PORT = 8086
DB_NAME = 'meteorology'
DB_USER = 'root'
DB_PASSWORD = 'root'
#stations = ['mythenquai', 'tiefenbrunnen']

### DataFrameClient and helper functions

In [792]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier


def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier



def get_all_historic_data(station):
    """
    Query all historic data
    :returns: pandas dataframe object
    """

    client = influxdb.DataFrameClient(DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME)

    query = """SELECT 
            *
            FROM "meteorology"."autogen"."mythenquai"
            WHERE time > now() - 4200d""".format(station)
    
    query = """SELECT 
                mean(air_temperature) AS air_temperature
                FROM "meteorology"."autogen"."mythenquai"
                WHERE time < now() GROUP BY time(1h)""".format(station)

    df = pd.DataFrame(client.query(query)[station])
    return df




In [782]:
dataframeli = get_all_historic_data("mythenquai")
dataframeli.tail()

,air_temperature
2019-12-24 11:00:00+00:00,4.866667
2019-12-24 12:00:00+00:00,5.850000
2019-12-24 13:00:00+00:00,6.783333
2019-12-24 14:00:00+00:00,7.816667
2019-12-24 15:00:00+00:00,8.500000


In [783]:
df_at = dataframeli["air_temperature"]

# resample data to 1 hour with mean calculation and round them
#df_at_forecast = (df_at.copy().resample('60min').mean()).round(1)

pd.set_option('display.max_rows', df_at_forecast.shape[0]+1)


air_temperature_last = df_at.tail(1)
#air_temperature_last = df_at_forecast.iloc[-1:]

#print('air_temperature_last: {}'.format(air_temperature_last))

hour_of_entry_last = air_temperature_last.index.hour[0]
time_of_entry_last = air_temperature_last.index.time[0]
hour_of_entry_1h_ago = int((air_temperature_last.index[0] + dt.timedelta(hours=-1)).strftime('%-H'))
hour_of_entry_2h_ago = int((air_temperature_last.index[0] + dt.timedelta(hours=-2)).strftime('%-H'))
hour_of_entry_4h_ago = int((air_temperature_last.index[0] + dt.timedelta(hours=-4)).strftime('%-H'))
hour_of_entry_7h_ago = int((air_temperature_last.index[0] + dt.timedelta(hours=-7)).strftime('%-H'))

print('hour_of_entry_last: {}'.format(hour_of_entry_last))
print('time_of_entry_last: {}'.format(time_of_entry_last))
print('hour_of_entry_1h_ago: {}'.format(hour_of_entry_1h_ago))
print('hour_of_entry_2h_ago: {}'.format(hour_of_entry_2h_ago))
print('hour_of_entry_4h_ago: {}'.format(hour_of_entry_4h_ago))
print('hour_of_entry_7h_ago: {}'.format(hour_of_entry_7h_ago))

mean_air_temperature_last = df_at_forecast.iloc[-1:]
mean_air_temperature_1h_ago = df_at_forecast.iloc[-2:-1]
mean_air_temperature_2h_ago = df_at_forecast.iloc[-3:-2]
mean_air_temperature_4h_ago = df_at_forecast.iloc[-5:-4]
mean_air_temperature_7h_ago = df_at_forecast.iloc[-8:-7]

print('mean_air_temperature_last: {}'.format(mean_air_temperature_last.values))
print('mean_air_temperature_1h_ago: {}'.format(mean_air_temperature_1h_ago.values))
print('mean_air_temperature_2h_ago: {}'.format(mean_air_temperature_2h_ago.values))
print('mean_air_temperature_4h_ago: {}'.format(mean_air_temperature_4h_ago.values))
print('mean_air_temperature_7h_ago: {}'.format(mean_air_temperature_7h_ago.values))



hour_of_entry_last: 15
time_of_entry_last: 15:00:00
hour_of_entry_1h_ago: 14
hour_of_entry_2h_ago: 13
hour_of_entry_4h_ago: 11
hour_of_entry_7h_ago: 8
mean_air_temperature_last: [6.4]
mean_air_temperature_1h_ago: [6.4]
mean_air_temperature_2h_ago: [6.5]
mean_air_temperature_4h_ago: [6.7]
mean_air_temperature_7h_ago: [7.]


In [789]:

def get_single_column(column_name, back_from_now):
    """
    Query single column
    :params column_name: column_name to query
    :params back_from_now: define time span for historic data
    :returns: pandas dataframe object
    """

    client = influxdb.DataFrameClient(DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME)

    query = """SELECT 
                {}
                FROM "meteorology"."autogen"."mythenquai" WHERE time >= now() - {}""".format(column_name, back_from_now)

    df = pd.DataFrame(client.query(query)["mythenquai"])
    return df



def get_mean_air_temperature_of_last_week_in_specific_timespan(back_from_now, time_start, time_end):
    df = get_single_column("air_temperature", back_from_now)
    df = df.between_time(time_start, time_end)
    df = df.mean()

    return df

a = get_mean_air_temperature_of_last_week_in_specific_timespan("7d", time_of_entry_last, time_of_entry_last)

mean_air_temperature_of_last_week_between_time = Decimal(str(float(a.values)))

print(format(mean_air_temperature_of_last_week_between_time, '.1f'))
print(format(9.0, '.1f'))


9.1
9.0


In [685]:

# Forecast air temperature with historic data
# Forecast the air temperature for 1 hour, 2 hours and 4 hours
air_temperature_forecasts = np.array(["N/A", "N/A", "N/A"])

# create copy of air temperature dataframe and resample it by hour
#df_at_forecast = df_at.copy()
#df_at_forecast = df_at_forecast.resample('H')
#df_at_forecast = df_at_forecast.rolling(5).mean()




# Select data in current timeframe per year: +/- 15 days
days_span = 10 # days to go back and forward
month_back = int((mean_air_temperature_last.index[0] + dt.timedelta(days=-days_span)).strftime('%-m'))
month_forward = int((mean_air_temperature_last.index[0] + dt.timedelta(days=days_span)).strftime('%-m'))
day_back = int((mean_air_temperature_last.index[0] + dt.timedelta(days=-days_span)).strftime('%-d'))
day_forward = int((mean_air_temperature_last.index[0] + dt.timedelta(days=days_span)).strftime('%-d'))
print('month_back: {}'.format(month_back))
print('day_back: {}'.format(day_back))
print('month_forward: {}'.format(month_forward))
print('day_forward: {}'.format(day_forward))
print('---')

df_at_filter = df_at_forecast[
    ((df_at_forecast.index.month >= month_back) & (df_at_forecast.index.day >= day_back)
    |
    (df_at_forecast.index.month <= month_forward) & (df_at_forecast.index.day <= day_forward))
]


#print(df_at_forecast.shape)
#print(df_at_forecast.tail(10))

# Select data in current timeframe: -8 hours from current time, +4 hours from current time
df_at_filter = df_at_filter.between_time(
    ((df_at_filter.tail(1).index + dt.timedelta(hours=-8)).strftime('%H:%M'))[0],
    ((df_at_filter.tail(1).index).strftime('%H:%M'))[0]
)

#print(df_at_forecast.shape)
#print(df_at_forecast.head(50))

#for items in df_at_forecast.iteritems(): 
    #print(items.day) 

_temp_timestamps = []


    
for index, value in df_at_filter.items():

    if index.hour >= (hour_of_entry_7h_ago - 2):
        
        # check values in span of +/- 1.5
        if float(value) >= float(mean_air_temperature_7h_ago - 1.5) and float(value) <= float(mean_air_temperature_7h_ago + 1.5):
            _temp_timestamps.append(index)

            
# check if data was found            
if len(_temp_timestamps) > 0:
    
    df_at_filter = df_at_filter[df_at_filter.index.isin(_temp_timestamps)]
    _temp_timestamps = []

    for index, value in df_at_filter.items():
        if index.hour >= (hour_of_entry_4h_ago - 1):

            # check values in span of +/- 1
            if float(value) >= float(mean_air_temperature_4h_ago - 1) and float(value) <= float(mean_air_temperature_4h_ago + 1):
                _temp_timestamps.append(index)

                
# check if data was found                          
if len(_temp_timestamps) > 0:
    
    df_at_filter = df_at_filter[df_at_filter.index.isin(_temp_timestamps)]
    _temp_timestamps = []

    for index, value in df_at_filter.items():
        if index.hour >= (hour_of_entry_2h_ago - 1):

            # check values in span of +/- 0.5
            if float(value) >= float(mean_air_temperature_2h_ago - 0.5) and float(value) <= float(mean_air_temperature_2h_ago + 0.5):
                _temp_timestamps.append(index)

                
# check if data was found                
if len(_temp_timestamps) > 0:
    
    df_at_filter = df_at_filter[df_at_filter.index.isin(_temp_timestamps)]
    _temp_timestamps = []

    for index, value in df_at_filter.items():
        if index.hour >= (hour_of_entry_1h_ago - 1) and index.hour <= (hour_of_entry_1h_ago + 1):

            # check values in span of +/- 0.25
            if float(value) >= float(mean_air_temperature_1h_ago - 0.25) and float(value) <= float(mean_air_temperature_1h_ago + 0.25):
                _temp_timestamps.append(index)            
                
    
print('len of _temp_timestamps: {}'.format(len(_temp_timestamps)))
print('len of df_at_forecast: {}'.format(len(df_at_forecast)))
print('---')
#print(_temp_timestamps)
    
df_at_relevant_day = df_at_filter.groupby(df_at_filter.index.floor('d')).size().reset_index(name='count')
print('df_at_relevant_day: {}'.format(df_at_relevant_day))
similar_day = str(df_at_relevant_day.nlargest(1, 'count')['index'])
print('similar_day: {}'.format(similar_day))

#df_at_used_day_for_forecast = df_at_forecast.isin([similar_day])
#print(df_at_used_day_for_forecast)





month_back: 12
day_back: 12
month_forward: 1
day_forward: 1
---
len of _temp_timestamps: 0
len of df_at_forecast: 100801
---
df_at_relevant_day:                        index  count
0  2008-12-20 00:00:00+00:00      9
1  2008-12-21 00:00:00+00:00      7
2  2008-12-22 00:00:00+00:00      6
3  2008-12-23 00:00:00+00:00      2
4  2009-12-22 00:00:00+00:00      1
5  2009-12-29 00:00:00+00:00      6
6  2009-12-31 00:00:00+00:00      4
7  2011-12-13 00:00:00+00:00      4
8  2011-12-14 00:00:00+00:00      5
9  2011-12-16 00:00:00+00:00      3
10 2011-12-22 00:00:00+00:00      5
11 2011-12-23 00:00:00+00:00      5
12 2011-12-31 00:00:00+00:00      2
13 2012-01-01 00:00:00+00:00      3
14 2012-12-15 00:00:00+00:00      1
15 2012-12-16 00:00:00+00:00      3
16 2012-12-17 00:00:00+00:00      3
17 2012-12-21 00:00:00+00:00      2
18 2012-12-22 00:00:00+00:00      1
19 2012-12-23 00:00:00+00:00      2
20 2012-12-24 00:00:00+00:00      2
21 2012-12-25 00:00:00+00:00      1
22 2012-12-26 00:00:00+00:0

In [556]:



# resample by hour and calculate mean


print(df_at_forecast.shape)
print(df_at_forecast.tail(100))



# Select data in current air temperature range: +/- 1 degree
#df_in_current_at_range = df_at_forecast[(df_at_forecast.values > round_down(air_temperature_last)) & (df_at_forecast.values < round_up(air_temperature_last))]

df_at_forecast = df_at_forecast[
    ((df_at_forecast.values > round_down(air_temperature_last[0] - 0.5))
        &
    (df_at_forecast.values < round_up(air_temperature_last[0] + 0.5))
        &
    (df_at_forecast.index.hour <= hour_of_entry_last))
    &
    ((df_at_forecast.values > round_down(air_temperature_last[0] - 2))
        &
    (df_at_forecast.values < round_up(air_temperature_last[0] + 2))
        &
    (df_at_forecast.index.hour <= hour_of_entry_1h_ago))
]


print(df_at_forecast.shape)
#print(df_at_forecast.head(1000))


#print(df_at_forecast.values.mean())




# result = data[(data['timestamp_cet'].dt.month == now.month) & (data['timestamp_cet'].dt.day == now.day) & (



# check for data after filtering (will return 0 if temperature is out of normal range)
# TODO: handle extreme lows or highs
#if df_at_forecast.shape[0] > 0:

    #print('df_at_forecast after filtering')
    #print(df_at_forecast.shape[0])

    #df_at_forecast = df_at_forecast[]

(2655,)
2019-12-11 17:00:00+00:00    False
2019-12-12 09:00:00+00:00    False
2019-12-12 10:00:00+00:00    False
2019-12-12 11:00:00+00:00    False
2019-12-12 12:00:00+00:00    False
2019-12-12 13:00:00+00:00    False
2019-12-12 14:00:00+00:00    False
2019-12-12 15:00:00+00:00    False
2019-12-12 16:00:00+00:00    False
2019-12-12 17:00:00+00:00    False
2019-12-13 09:00:00+00:00    False
2019-12-13 10:00:00+00:00    False
2019-12-13 11:00:00+00:00    False
2019-12-13 12:00:00+00:00    False
2019-12-13 13:00:00+00:00    False
2019-12-13 14:00:00+00:00    False
2019-12-13 15:00:00+00:00    False
2019-12-13 16:00:00+00:00    False
2019-12-13 17:00:00+00:00    False
2019-12-14 09:00:00+00:00    False
2019-12-14 10:00:00+00:00    False
2019-12-14 11:00:00+00:00    False
2019-12-14 12:00:00+00:00    False
2019-12-14 13:00:00+00:00    False
2019-12-14 14:00:00+00:00    False
2019-12-14 15:00:00+00:00    False
2019-12-14 16:00:00+00:00    False
2019-12-14 17:00:00+00:00    False
2019-12-15 0